In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [ ]:
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="こんにちは！私は東京に住んでいます。")]},
    config
):
    print(chunk)
    print("----")

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="私が住んでいる場所の天気は？")]},
    config
):
    print(chunk)
    print("----")

In [ ]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("LLMエージェントとはなんですか？")
print(search_results)

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="LLMエージェントとは？")])
print(response.content)

In [ ]:
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="LLMエージェントとは？")])

In [ ]:
print(response.content)
print(response.tool_calls)

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="こんにちは")]})
print(response["messages"])

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="1ドル何円ですか？")]}
)
print(response["messages"])

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="LLMエージェントとは？")]}
):
    print(chunk)
    print("-----")

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="LLMエージェントとは？")]},
    version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

In [ ]:
memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="こんにちは。私はボブです。")]},
    config
):
    print(chunk)
    print("-----")

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="私の名前は？")]},
    config
):
    print(chunk)
    print("-----")